In [25]:
# Importa a biblioteca Pandas, que é a nossa principal ferramenta para trabalhar com dados.
# Usamos 'as pd' como um apelido para facilitar a escrita do código.
import pandas as pd

In [26]:
# Carregamos o arquivo CSV para um DataFrame, lendo apenas 1 milhão de linhas para agilizar a análise.
# Um DataFrame é como uma planilha inteligente dentro do Python.
df = pd.read_csv('dados/2019-Oct.csv', nrows=1000000)

In [27]:
# O comando .info() nos dá um resumo técnico de cada coluna: o nome, a quantidade de valores não-nulos e o tipo de dado.
# É o nosso "raio-x" inicial para encontrar problemas.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   event_time     1000000 non-null  object 
 1   event_type     1000000 non-null  object 
 2   product_id     1000000 non-null  int64  
 3   category_id    1000000 non-null  int64  
 4   category_code  681869 non-null   object 
 5   brand          852440 non-null   object 
 6   price          1000000 non-null  float64
 7   user_id        1000000 non-null  int64  
 8   user_session   1000000 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 68.7+ MB


In [28]:
# A coluna 'event_time' é lida como texto ('object'), mas precisamos que ela seja uma data para análises de tempo.
# O pd.to_datetime() faz essa conversão.
df['event_time'] = pd.to_datetime(df['event_time'])

# As colunas 'category_code' e 'brand' têm valores faltando. Preenchemos com 'unknown' para não perdermos os dados.
df['category_code'].fillna('unknown', inplace=True)
df['brand'].fillna('unknown', inplace=True)

# Verificamos novamente com .info() para confirmar que a limpeza funcionou.
print("\n--- Dados após a limpeza ---")
df.info()


--- Dados após a limpeza ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype              
---  ------         --------------    -----              
 0   event_time     1000000 non-null  datetime64[ns, UTC]
 1   event_type     1000000 non-null  object             
 2   product_id     1000000 non-null  int64              
 3   category_id    1000000 non-null  int64              
 4   category_code  1000000 non-null  object             
 5   brand          1000000 non-null  object             
 6   price          1000000 non-null  float64            
 7   user_id        1000000 non-null  int64              
 8   user_session   1000000 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(3), object(4)
memory usage: 68.7+ MB


/var/folders/2b/jqf6xbrd60lgmz6q7j4511jh0000gn/T/ipykernel_8775/2540778922.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['category_code'].fillna('unknown', inplace=True)
/var/folders/2b/jqf6xbrd60lgmz6q7j4511jh0000gn/T/ipykernel_8775/2540778922.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw

In [29]:
# Agrupamos os dados por tipo de evento ('view', 'cart', 'purchase') e contamos quantos usuários únicos existem em cada etapa.
usuarios_por_etapa = df.groupby('event_type')['user_id'].nunique()
print("--- Usuários únicos por etapa do funil ---")
print(usuarios_por_etapa)

# Agora, calculamos as taxas de conversão para entender a eficiência do e-commerce.
total_view = usuarios_por_etapa.get('view', 0)
total_cart = usuarios_por_etapa.get('cart', 0)
total_purchase = usuarios_por_etapa.get('purchase', 0)

taxa_view_para_cart = (total_cart / total_view) * 100
taxa_cart_para_purchase = (total_purchase / total_cart) * 100

print(f"\nTaxa de Conversão (Visualização para Carrinho): {taxa_view_para_cart:.2f}%")
print(f"Taxa de Conversão (Carrinho para Compra): {taxa_cart_para_purchase:.2f}%")

--- Usuários únicos por etapa do funil ---
event_type
cart          7665
purchase     12452
view        162994
Name: user_id, dtype: int64

Taxa de Conversão (Visualização para Carrinho): 4.70%
Taxa de Conversão (Carrinho para Compra): 162.45%


In [30]:
# Filtramos o DataFrame para pegar apenas os eventos de 'purchase' (compra).
df_compras = df[df['event_type'] == 'purchase']

# Contamos quais categorias de produtos aparecem mais vezes na nossa lista de compras.
top_10_categorias_compradas = df_compras['category_code'].value_counts().head(10)
print("\n--- Top 10 Categorias Mais Compradas ---")
print(top_10_categorias_compradas)


--- Top 10 Categorias Mais Compradas ---
category_code
electronics.smartphone               7930
unknown                              3583
electronics.audio.headphone           772
electronics.video.tv                  443
electronics.clocks                    419
computers.notebook                    408
appliances.kitchen.washer             300
appliances.environment.vacuum         278
appliances.kitchen.refrigerators      221
appliances.environment.air_heater     132
Name: count, dtype: int64


In [31]:
# Salvamos nossos resultados em novos arquivos CSV. Estes arquivos serão a fonte de dados para o nosso dashboard.
usuarios_por_etapa.to_csv('dados_funil_etapas.csv')
top_10_categorias_compradas.to_csv('dados_top_categorias.csv')

print("\nArquivos CSV para o dashboard foram gerados com sucesso!")


Arquivos CSV para o dashboard foram gerados com sucesso!
